## JAX As Accelerated NumPy

### JAX NumPy 시작하기

- 기본적으로 JAX는 배열 조작 프로그램의 변환을 가능하게 해주는 NumPy-like한 API
- 현재로써 JAX는 accelerator로 구동 가능한 NumPy라고 생각하면 됨

In [1]:
import jax
import jax.numpy as jnp

In [2]:
x = jnp.arange(10)
print(x)

2024-02-26 22:29:56.676253: W external/xla/xla/service/gpu/nvptx_compiler.cc:744] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


[0 1 2 3 4 5 6 7 8 9]


- JAX의 가장 큰 장점은 새로이 API를 배울 필요가 없다는 것
- 일반적으로 NumPy 프로그램은 np를 jnp로 대체하는 경우 JAX에서도 잘 실행됨
- 몇 가지 중요한 차이점은 마지막 부분에 설명

In [3]:
x

Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

- JAX의 유용한 기능 중 하나는 동일한 코드를 CPU, GPU, TPU 등 다양한 백엔드에서 실행할 수 있다는 것
- Technical detail
    - JAX 함수가 호출될 때 해당 연산이 가능한 경우, 비동기적으로 계산될 수 있도록 accelerator로 전송됨
    - 따라서, 반환된 배열은 함수가 반환되는 즉시 '채워지는(filled in)' 것은 아님
    - 결과가 즉시 필요하지 않은 경우, 연산이 파이썬의 execution을 즉시 차단하지 않음
    - `block_until_ready`를 실행하거나 일반적 Python type으로 배열을 변환하지 않는 한 실제 연산 시간이 아닌 dispatch 시간만 계산됨

In [4]:
long_vector = jnp.arange(int(1e7))
%timeit jnp.dot(long_vector, long_vector).block_until_ready()

1.25 ms ± 4.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### 첫 번째 JAX 기반 변환: `grad`

- JAX의 기본 기능은 함수를 변환하는 것
- 가장 일반적으로 사용되는 변환 중 하나는 ㅍ파이썬으로 작성된 수치 함수를 가져와 원래 함수의 기울기를 계산하는 `jax.grad`

In [5]:
def sum_of_squares(x):
    return jnp.sum(x**2)

- `jax.grad`를 `sum_of_squares`에 적용하면 첫 번째 매개변수 `x`에 대한 `sum_of_squares`의 기울기를 반환
- 그런 다음 배열에서 해당 함수를 사용해 배열의 각 요소에 대한 도함수를 반환할 수 있음

In [6]:
sum_of_squares_dx = jax.grad(sum_of_squares)

x = jnp.asarray([1.0, 2.0, 3.0, 4.0])

print(sum_of_squares(x))
print(sum_of_squares_dx(x))

30.0
[2. 4. 6. 8.]


- $\nabla$와 같이 `jax.grad`는 스칼라 출력이 있는 함수에서만 작동하며, 그렇지 않은 경우 에러 발생
- 따라서, JAX API는 loss tensor 자체를 사용해 기울기를 계산하는 (e.g., `loss.backward()`) 다른 자동미분 라이브러리와는 상당히 다름
- JAX API는 함수를 직접 사용해 기본적인 '수학'에 조금 더 가깝게 작동함
    - 여기에 익숙해지면, loss function이 실제로 매개변수와 데이터의 '함수'이며, 수학에서와 마찬가지로 기울기를 찾을 수 있다는 점이 자연스럽게 느껴질 것
    - 이러한 방식을 사용하면 어떤 변수를 기준으로 미분할지 등을 간단히 조작할 수 있음

In [8]:
def sum_squared_error(x, y):
    return jnp.sum((x - y)**2)

In [10]:
sum_squared_error_dx = jax.grad(sum_squared_error)

y = jnp.asarray([1.1, 2.1, 3.1, 4.1])

print(sum_squared_error_dx(x, y))

[-0.20000005 -0.19999981 -0.19999981 -0.19999981]


- 다른 여러 argument에 대한 기울기를 찾고 싶다면 `argnums`를 설정하면 됨

In [12]:
display(jax.grad(sum_squared_error, argnums=(0, 1))(x, y)) # x와 y에 대한 편미분 값을 모두 반환

(Array([-0.20000005, -0.19999981, -0.19999981, -0.19999981], dtype=float32),
 Array([0.20000005, 0.19999981, 0.19999981, 0.19999981], dtype=float32))

- 그렇다면, ML 코드를 짤 때 거대한 매개변수 배열에 대해 거대한 argument list로 함수를 작성해야 할까?
- JAX에는 '`pytree`'라는 메커니즘이 탑재되어 있어 데이터 구조로 배열을 함께 묶을 수 있음
- 따라서 `jax.grad`는 다음과 같이 사용할 수 있음
    ```python
    def loss_fn(params, data):
        ...

    grads = jax.grad(loss_fn)(params, data_batch)
    ```